## Import 'vehicles' Dataframe

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\skyla\Downloads\vehicles.csv")

In [3]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

## Drop Columns

In [4]:
#URL doesn't add anything to our dataset
df = df.drop('url', axis=1)

In [5]:
#Region URL doesn't add anything to our dataset
df = df.drop('region_url', axis=1)

In [6]:
#Image URL doesn't add anything to our dataset
df = df.drop('image_url', axis=1)

In [7]:
#This is a useful column but slows down the dataset too much. 
#I don't have the hardware or time to realistically transform and analyse this column.
df = df.drop('description', axis=1)

In [8]:
#Latitude is out of scope for analysis 
df = df.drop('lat', axis=1)

In [9]:
#Longitude is out of scope for analysis 
df = df.drop('long', axis=1)

In [10]:
#Too many null values and no way to impute
df = df.drop('paint_color', axis=1)

In [11]:
# Filter out rows where 'price' is NaN or 'price' equals 0
df = df.dropna(subset=['price'])
df = df[df['price'] != 0]

In [12]:
# Calculate the threshold for columns to keep (at least 70% data present)
threshold_columns = len(df) * 0.7

# Remove columns with more than 30% missing values
df = df.dropna(thresh=threshold_columns, axis=1)

In [13]:
# Calculate the threshold for rows to keep (at least 70% data present)
threshold_rows = len(df.columns) * 0.7

# Remove rows with more than 30% missing values
df = df.dropna(thresh=threshold_rows, axis=0)

## Impute Date Column

In [14]:
#Convert Posting Date to DateTime format
df['posting_date'] = pd.to_datetime(df['posting_date'], utc=True, format='%Y-%m-%dT%H:%M:%S%z')

## Descriptive Statistics

In [15]:
df.shape

(393443, 13)

In [16]:
df.columns

Index(['id', 'region', 'price', 'year', 'manufacturer', 'model', 'fuel',
       'odometer', 'title_status', 'transmission', 'type', 'state',
       'posting_date'],
      dtype='object')

In [17]:
#Columns that have been removed are 
#url
#region_url
#condition
#cylinders
#VIN
#drive
#size
#image_url
#county

In [18]:
df.dtypes

id                            int64
region                       object
price                         int64
year                        float64
manufacturer                 object
model                        object
fuel                         object
odometer                    float64
title_status                 object
transmission                 object
type                         object
state                        object
posting_date    datetime64[ns, UTC]
dtype: object

In [19]:
# Display the first few rows of the 'posting_date' to verify the conversion
print(df['posting_date'].head())

27   2021-05-04 17:31:18+00:00
28   2021-05-04 17:31:08+00:00
29   2021-05-04 17:31:25+00:00
30   2021-05-04 15:41:31+00:00
31   2021-05-03 19:02:03+00:00
Name: posting_date, dtype: datetime64[ns, UTC]


In [20]:
# Extract date features
df['posting_year'] = df['posting_date'].dt.year
df['posting_month'] = df['posting_date'].dt.month
df['posting_day'] = df['posting_date'].dt.day
df['posting_weekday'] = df['posting_date'].dt.weekday  # Monday=0, Sunday=6
df['posting_hour'] = df['posting_date'].dt.hour

In [21]:
# Display the first few rows of the DataFrame to inspect the new date-related columns
print(df[['posting_date', 'posting_year', 'posting_month', 'posting_day', 'posting_weekday', 'posting_hour']].head())

                posting_date  posting_year  posting_month  posting_day  \
27 2021-05-04 17:31:18+00:00          2021              5            4   
28 2021-05-04 17:31:08+00:00          2021              5            4   
29 2021-05-04 17:31:25+00:00          2021              5            4   
30 2021-05-04 15:41:31+00:00          2021              5            4   
31 2021-05-03 19:02:03+00:00          2021              5            3   

    posting_weekday  posting_hour  
27                1            17  
28                1            17  
29                1            17  
30                1            15  
31                0            19  


## Impute Columns

In [22]:
#This is to fill in missing 'Manufacturer' values using 'Model' values, identical model values
#That are matching, are used to fill in 'Manufacturer' using the Mean 'Model' value that matches
# Create a dictionary to store mapping of models to most frequent manufacturers
model_to_most_frequent_manufacturer = {}

# Populate the dictionary with the most frequent 'manufacturer' values for each 'model'
for model, group in df.groupby('model'):
    if not group['manufacturer'].isnull().all():
        most_frequent_manufacturer = group['manufacturer'].mode().iloc[0]
        model_to_most_frequent_manufacturer[model] = most_frequent_manufacturer

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if the 'manufacturer' value is empty for the current row
    if pd.isnull(row['manufacturer']):
        # Get the 'model' value for the current row
        model = row['model']
        # If the 'model' is in the dictionary, fill in the missing 'manufacturer' value
        if model in model_to_most_frequent_manufacturer:
            df.at[index, 'manufacturer'] = model_to_most_frequent_manufacturer[model]

In [23]:
print(df['model'].head())

27    sierra 1500 crew cab slt
28              silverado 1500
29         silverado 1500 crew
30        tundra double cab sr
31                   f-150 xlt
Name: model, dtype: object


In [24]:
def extract_base_model(model):
    # Check if the model is a string; if not, convert it to string
    # This also handles NaN and None values by converting them to the string 'nan'
    base_model = str(model).split()[0]
    return base_model

# Apply the function to the 'model' column to create a new 'base_model' column
df['base_model'] = df['model'].apply(extract_base_model)

# Display the result to verify
print(df[['model', 'base_model']].head())

                       model base_model
27  sierra 1500 crew cab slt     sierra
28            silverado 1500  silverado
29       silverado 1500 crew  silverado
30      tundra double cab sr     tundra
31                 f-150 xlt      f-150


In [25]:
# Count the rows where 'base_model' is purely numeric
numeric_base_models_count = df[df['base_model'].str.isdigit()].shape[0]

# Print the count
print(f"Number of purely numeric 'base_model' entries: {numeric_base_models_count}")

Number of purely numeric 'base_model' entries: 26411


In [26]:
df.shape

(393443, 19)

In [27]:
# Remove rows where 'base_model' is purely numeric
df = df[~df['base_model'].str.isdigit()]

# Display the result or check the shape to confirm removal
print(df.shape)

(367032, 19)


In [28]:
# Count the number of unique 'base_model' values in the filtered DataFrame
unique_base_models_count = df['base_model'].nunique()

# Print the count
print(f"Number of unique 'base_model' values: {unique_base_models_count}")

Number of unique 'base_model' values: 4690


In [29]:
#Replacing this with Base Model
df = df.drop('model', axis=1)

In [30]:
#We now have the aggregated values and can remove this to simply our dataset
df = df.drop('posting_date', axis=1)

In [31]:
unique_base_models = df['base_model'].unique()
unique_base_models_df = pd.DataFrame(unique_base_models, columns=['base_model'])
unique_base_models_df.to_csv('unique_base_models.csv', index=False)

In [32]:
#Getting rid of some of the unreadable values
# List of strings to remove from base_model values
strings_to_remove = [
    "-", "%", "-2018", "2-Jan", "3-Sep", "4-Mar", "155,037", "7.3", "3.2", "1.5",
    "198.5", "3.5", "(cng)", "*matrix*", "*", "2.5", "4.6", "-350", "99.5", "2003",
    "-210", "-300", "135,826", "t", "(s)port", "2.4", "-150", "1986", "$362.47", "&", "2",
    "#NAME?", ":", "/", "//", "45538", "45293", '.', '2003', '1986', '2', '9/3/2024', '1/2/2024'
]

# Define the list of IDs to delete base_model values for
ids_to_delete = [7302049019, 7303726176, 7303889514, 7304172920, 7304766070, 7306374142,
7307179181, 7307984950, 7308311655, 7308436673, 7310003852, 7310724097,
7310905114, 7311156124, 7312639238, 7312807312, 7313478534, 7313615145,
7314081937, 7314348336, 7314500907, 7315012429, 7315076544, 7315165698,
7315229169, 7315376783, 7315377882, 7315378199, 7315379051, 7315514859,
7316150205, 7316150889, 7316181280, 7316181424, 7316450659, 7316525434,
7316572113, 7316803175]



# Remove base_model values containing specified strings
df = df[~df['base_model'].isin(strings_to_remove)]

# Remove rows with specific values from the DataFrame
df = df[~df['manufacturer'].isin(['45538', '45293', '.', '2003', '1986', '2', '9/3/2024', '1/2/2024'])]

# Delete the base_model values for the specified IDs
df.loc[df['id'].isin(ids_to_delete), 'base_model'] = None

In [33]:
# Calculate Q1 and Q3
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)

# Calculate the IQR
IQR = Q3 - Q1

# Adjust the multiplier for the upper bound to make it more lenient
# and set a minimum price to remove unrealistically low prices
multiplier = 5  # Adjusting this value can make the filter more lenient or strict
lower_bound = Q1 - 1.5 * IQR  # You might keep the lower bound less changed
upper_bound = Q3 + multiplier * IQR  # Increase the upper bound to be more lenient

# Define a realistic minimum price to filter out $1 prices
minimum_price = 100  # Example minimum price, adjust as needed

# Remove outliers with adjusted criteria and save back to df
df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound) & (df['price'] > minimum_price)]

In [34]:
# Assuming df is your DataFrame
# This line replaces any characters at the start of the string that are not letters or numbers with an empty string
df['base_model'] = df['base_model'].str.replace('^[^a-zA-Z0-9]*', '', regex=True)

In [35]:
# Step 1: Identify all unique, non-empty manufacturer names
unique_manufacturers = df['manufacturer'].dropna().unique()

# Step 2: Normalize the 'base_model' and manufacturer names for comparison
df['normalized_base_model'] = df['base_model'].str.lower()

# Step 3: For each unique manufacturer, try to find and fill missing manufacturer values based on 'base_model'
for manufacturer in unique_manufacturers:
    normalized_manufacturer = manufacturer.lower()
    mask = df['manufacturer'].isna() & df['normalized_base_model'].str.contains(normalized_manufacturer)
    df.loc[mask, 'manufacturer'] = manufacturer

# Optional: Clean up by removing the temporary 'normalized_base_model' column
df.drop('normalized_base_model', axis=1, inplace=True)

# This code block goes through each row where 'manufacturer' is missing and checks if 'base_model' (ignoring case)
# contains a string that matches one of the unique, non-missing manufacturer names. If a match is found,
# it fills in the 'manufacturer' column with the corresponding manufacturer name.

In [36]:
#Remove Odometer Upper Limit Outliars
Q1 = df['odometer'].quantile(0.25)
Q3 = df['odometer'].quantile(0.75)
IQR = Q3 - Q1
upper_limit = Q3 + 1.5 * IQR

df = df[df['odometer'] <= upper_limit]

In [37]:
# Remove Year Lower Limit Outliers
Q1_year = df['year'].quantile(0.25)
Q3_year = df['year'].quantile(0.75)
IQR_year = Q3_year - Q1_year
lower_limit_year = Q1_year - 1.5 * IQR_year

df = df[df['year'] >= lower_limit_year]

In [38]:
# Create a mapping from base_model to manufacturer based on non-null entries in the dataframe
valid_manufacturer_base_model = df.dropna(subset=['manufacturer', 'base_model'])
base_model_to_manufacturer = valid_manufacturer_base_model.drop_duplicates(subset=['base_model'])[['base_model', 'manufacturer']].set_index('base_model')['manufacturer'].to_dict()

# Function to apply the mapping to fill missing manufacturer values
def fill_manufacturer(row):
    if pd.isnull(row['manufacturer']) and row['base_model'] in base_model_to_manufacturer:
        return base_model_to_manufacturer[row['base_model']]
    else:
        return row['manufacturer']

# Apply the function to fill missing 'manufacturer' values based on 'base_model'
df['manufacturer'] = df.apply(fill_manufacturer, axis=1)

# Optionally, to check how many 'manufacturer' values are still missing after this operation
missing_manufacturer_after = df['manufacturer'].isnull().sum()
print(f"Missing 'manufacturer' values after filling: {missing_manufacturer_after}")

Missing 'manufacturer' values after filling: 8714


In [39]:
# Remove all rows where 'base_model' is null
df = df.dropna(subset=['base_model'])

# Optionally, to check the new shape of the dataframe after removal
print(df.shape)

(343615, 17)


In [40]:
# Count null values in each column
null_values_count = df.isnull().sum()

print(null_values_count)

id                     0
region                 0
price                  0
year                   0
manufacturer        8705
fuel                1887
odometer               0
title_status        6513
transmission        1385
type               70898
state                  0
posting_year           0
posting_month          0
posting_day            0
posting_weekday        0
posting_hour           0
base_model             0
dtype: int64


In [41]:
# Define a mapping of known base models or model prefixes to manufacturers
model_prefix_to_manufacturer = {
    'sierra': 'gmc',
    'silverado': 'chevrolet',
    'tundra': 'toyota',
    'f-150': 'ford',
    'f150': 'ford',
    'tacoma': 'toyota',
    'corvette': 'chevrolet',
    'wrangler': 'jeep',
    'camaro': 'chevrolet',
    'ranger': 'ford',
    'frontier': 'nissan',
    'mx-5': 'mazda',
    'xt4': 'cadillac',
    'f250': 'ford',
    'renegade': 'jeep',
    'odyssey': 'honda',
    'mustang': 'ford',
    'f450': 'ford',
    'charger': 'dodge',
    # Continue adding more mappings as needed based on the dataset and common knowledge
}

# Function to infer manufacturer from base_model using the mapping
def infer_manufacturer_from_base_model(base_model):
    base_model = base_model.lower()  # Ensure matching is case-insensitive
    for model_prefix, manufacturer in model_prefix_to_manufacturer.items():
        if model_prefix in base_model:
            return manufacturer
    return None

# Apply the function to fill missing 'manufacturer' values based on 'base_model'
df['manufacturer'] = df.apply(
    lambda row: infer_manufacturer_from_base_model(row['base_model']) if pd.isnull(row['manufacturer']) else row['manufacturer'], 
    axis=1
)

# Verify the application by checking the number of missing 'manufacturer' values after inference
missing_manufacturer_after_inference = df['manufacturer'].isnull().sum()

missing_manufacturer_after_inference

8547

In [42]:
# Known manufacturers list for matching
known_manufacturers = known_manufacturers = [
    'ford', 'chevrolet', 'toyota', 'honda', 'nissan', 'jeep', 'gmc', 'dodge', 
    'bmw', 'mercedes', 'audi', 'lexus', 'volkswagen', 'subaru', 'hyundai', 
    'kia', 'mazda', 'porsche', 'ferrari', 'lamborghini', 'tesla', 'volvo', 
    'mitsubishi', 'land rover', 'jaguar', 'buick', 'cadillac', 'chrysler', 
    'lincoln', 'infiniti', 'acura', 'alfa romeo', 'fiat', 'genesis', 'mini', 
    'suzuki', 'saab', 'scion', 'hummer', 'maserati', 'isuzu', 'hino', 'bentley', 
    'mack', 'smart', 'lotus'
]


# Function to infer manufacturer from base_model
def infer_manufacturer_from_base_model(base_model):
    if pd.isna(base_model) or not isinstance(base_model, str):
        return None
    base_model = base_model.lower()
    for manufacturer in known_manufacturers:
        if manufacturer in base_model:
            return manufacturer.capitalize()
    return None

# Infer and fill missing 'manufacturer' based on 'base_model'
df['manufacturer'] = df.apply(lambda row: infer_manufacturer_from_base_model(row['base_model']) if pd.isnull(row['manufacturer']) else row['manufacturer'], axis=1)

# Extract the updated mapping of 'base_model' to 'manufacturer'
updated_mapping = df[['base_model', 'manufacturer']].drop_duplicates().set_index('base_model')['manufacturer'].to_dict()

# Print a portion of the updated mapping
print({k: updated_mapping[k] for k in list(updated_mapping)[:10]})

{'sierra': 'chevrolet', 'silverado': 'chevrolet', 'tundra': 'toyota', 'f-150': 'ford', 'tacoma': 'toyota', 'colorado': 'chevrolet', 'corvette': 'chevrolet', 'wrangler': 'jeep', 'camaro': 'chevrolet', 'ranger': 'jeep'}


In [43]:
# Define a custom mapping based on the list and additional knowledge
custom_mapping = {
    'International': 'International Trucks',
    'Freightliner': 'Freightliner',
    'oldsmobile': 'Oldsmobile',
    'Sterling': 'Sterling Trucks',
    'Kenworth': 'Kenworth',
    'Workhorse': 'Workhorse',
    'hyndai': 'Hyundai',
    'chryler': 'Chrysler',
    'caddilac': 'Cadillac',
    'CHEVORLET': 'Chevrolet',
    'Volkswagon': 'Volkswagen',
    'Pierce': 'Pierce',
    'PETERBILT': 'Peterbilt',
    'ROLLS': 'Rolls-Royce',
    'YAMAHA': 'Yamaha',
    'Plymouth': 'Plymouth',
    'Corvette': 'Chevrolet',  # Corvette is a model by Chevrolet
    'Prius': 'Toyota',  # Prius is a model by Toyota
    # Add more mappings based on the list and internet research
}

# Normalize the base_model column to match keys in custom_mapping (case-insensitive)
df['base_model_normalized'] = df['base_model'].str.lower()

# Function to infer manufacturer from base_model using the custom mapping
def infer_manufacturer_custom(base_model_normalized, custom_mapping):
    for model, manufacturer in custom_mapping.items():
        if model.lower() == base_model_normalized:
            return manufacturer
    return None

# Apply the function to fill missing 'manufacturer' values based on the custom mapping
df['manufacturer'] = df.apply(
    lambda row: infer_manufacturer_custom(row['base_model_normalized'], custom_mapping) if pd.isnull(row['manufacturer']) else row['manufacturer'], 
    axis=1
)

# Drop the temporary normalized base_model column if no longer needed
df.drop('base_model_normalized', axis=1, inplace=True)

In [44]:
# Correctly define the custom mapping with new information from the updated list
custom_mapping2 = {
    'chysler': 'Chrysler',
    'SILVERADO': 'Chevrolet',
    'F250': 'Ford',
    'F150': 'Ford',
    'F-150': 'Ford',
    'Grand': 'Jeep',  # Assuming it refers to Grand Cherokee
    'POLARIS': 'Polaris',  # Manufacturer of ATVs and motorcycles, if relevant
    'WOLF': None,  # This might need clarification
    'Mercedez': 'Mercedes-Benz',  # Correcting typo
    'Silverado': 'Chevrolet',
    'VPG': 'VPG',  # Vehicle Production Group, known for MV-1
    'hundai': 'Hyundai',  # Correcting typo
    'Mustang': 'Ford',
    'cheverolet': 'Chevrolet',  # Correcting typo
    'Cheverolet': 'Chevrolet',
    'Wrangler': 'Jeep',
    'MB': 'Mercedes-Benz',
    'MV-1': 'VPG',
    'huyndai': 'Hyundai',  # Correcting typo
    'Camaro': 'Chevrolet',
    'F350': 'Ford',
    'mercedez': 'Mercedes-Benz',  # Correcting typo
    'Cadilac': 'Cadillac',  # Correcting typo
    'volkwagen': 'Volkswagen',  # Correcting typo
    'SL': 'Mercedes-Benz',  # Assuming it refers to the Mercedes SL-Class
    # Add or update mappings as needed
}

# Normalize the base_model column to match keys in custom_mapping (case-insensitive)
df['base_model_normalized'] = df['base_model'].str.lower()

# Function to infer manufacturer from base_model using the custom mapping
def infer_manufacturer_custom(base_model_normalized, custom_mapping2):
    for model, manufacturer in custom_mapping2.items():
        if model.lower() == base_model_normalized:
            return manufacturer
    return None

# Apply the function to fill missing 'manufacturer' values based on the custom mapping
df['manufacturer'] = df.apply(
    lambda row: infer_manufacturer_custom(row['base_model_normalized'], custom_mapping2) if pd.isnull(row['manufacturer']) else row['manufacturer'], 
    axis=1
)

# Drop the temporary normalized base_model column if no longer needed
df.drop('base_model_normalized', axis=1, inplace=True)

In [45]:
# Expand the custom_mapping2 with new information
custom_mapping3 = {
    # Correcting typos and mapping models to manufacturers
    'Gm': 'General Motors',
    'suburu': 'Subaru',  # Typo correction
    'Accord': 'Honda',  # Model to manufacturer
    'CHEV.': 'Chevrolet',  # Abbreviation to full name
    'CHEC': None,  # Unclear reference, needs clarification
    'Keep': 'Jeep',  # Possibly a typo for Jeep
    'PROMASTER': 'Ram',  # Model to manufacturer
    'Nissa': 'Nissan',  # Typo correction
    'chrystler': 'Chrysler',  # Typo correction
    'chrylser': 'Chrysler',  # Typo correction
    'PT': 'Chrysler',  # Referring to PT Cruiser
    'F800': None,  # Likely a model of Ford trucks, needs clarification
    'chevolet': 'Chevrolet',  # Typo correction
    'RANGER': 'Ford',  # Model to manufacturer
    'Yukon': 'GMC',  # Model to manufacturer
    'sierra': 'GMC',  # Model to manufacturer
    'mistubishi': 'Mitsubishi',  # Typo correction
    'Hyundia': 'Hyundai',  # Typo correction
    'Toyoya': 'Toyota',  # Typo correction
    # General terms or unclear references are set to None or handled specifically if possible
    'ALL': None,
    'All': None,
    'BUY': None,
    'IC': None,  # Could refer to IC Bus, needs clarification
    'Janesville': None,
    'Flexible': None,
    'Blue': None,
    'to': None,
    'SPECIAL': None,
    'cars': None,
    'Other': None,
    'Mobility': None,
    'Emergency': None,
    'any': None,
    'Any': None,
    'NEW': None,
    'blue': None,
    'WOLF': None,
    'Keystone': None,
    'THE': None,
    'Utilimaster': 'Utilimaster',  # Manufacturer of walk-in vans and commercial vehicles
    'WATER': None,
    'Club': None,
    'CLICK': None,
    'ASK': None,
    'FABRIQUE': None,
    'hiunday': 'Hyundai',
    'SANTA': 'Hyundai',  # Likely referring to Hyundai Santa Fe
    # Add more mappings or corrections as necessary
}

# Normalize the base_model column to match keys in custom_mapping (case-insensitive)
df['base_model_normalized'] = df['base_model'].str.lower()

# Function to infer manufacturer from base_model using the custom mapping
def infer_manufacturer_custom(base_model_normalized, custom_mapping3):
    for model, manufacturer in custom_mapping3.items():
        if model.lower() == base_model_normalized:
            return manufacturer
    return None

# Apply the function to fill missing 'manufacturer' values based on the custom mapping
df['manufacturer'] = df.apply(
    lambda row: infer_manufacturer_custom(row['base_model_normalized'], custom_mapping3) if pd.isnull(row['manufacturer']) else row['manufacturer'], 
    axis=1
)

# Drop the temporary normalized base_model column if no longer needed
df.drop('base_model_normalized', axis=1, inplace=True)

In [46]:
# Extend custom_mapping2 with more specific mappings and corrections
custom_mapping4 = {
    # Correcting specific vehicle models to their manufacturers
    'Tacoma': 'Toyota',
    'Civic': 'Honda',
    'F-350': 'Ford',
    'Jetta': 'Volkswagen',
    'Geo': 'Chevrolet',  # Geo was a brand under Chevrolet
    'Eagle': None,  # Eagle was a brand under Chrysler, now defunct
    'F-250': 'Ford',
    'nissian': 'Nissan',  # Typo correction
    'izusu': 'Isuzu',  # Typo correction
    'Kawasaki': 'Kawasaki',  # Manufacturer of motorcycles and ATVs, if relevant
    'BMX': None,  # Likely a typo for BMW, or referring to bicycles
    'FRHT': None,  # Unclear reference, possibly meant for Freightliner
    'SUPER': None,  # Too generic, possibly part of a model name
    'bluebird': 'Blue Bird Corporation',  # Manufacturer of school and activity buses
    'John': None,  # Could refer to John Deere, if agricultural vehicles are relevant
    '2D8HN54159R611084': None,  # This appears to be a VIN, not applicable for manufacturer inference
    # Handling generic or ambiguous terms by setting to None or applying specific logic if possible
    'ALL': None,
    'All': None,
    'BUY': None,
    'IC': 'IC Bus',  # Manufacturer of school buses and commercial buses
    'Janesville': None,  # Likely refers to a location, not a manufacturer
    'Flexible': None,
    'Blue': None,
    'to': None,
    'SPECIAL': None,
    'cars': None,
    'Other': None,
    'Mobility': None,
    'Emergency': None,
    'any': None,
    'Any': None,
    'NEW': None,
    'blue': None,
    'WOLF': None,
    'Keystone': None,
    'THE': None,
    'WATER': None,
    'Cars': None,
    'CHEC': None,
    'Club': None,
    'F800': 'Ford',  # F800 is a model of Ford trucks
    'Town': None,  # Possibly referring to Chrysler Town & Country
    'wheelchair': None,
    'todos': None,
    'CLICK': None,
    'Porshe': 'Porsche',
    'am': None,  # Too generic, needs clarification
    'Western': None,  # Could refer to Western Star Trucks if trucks are relevant
    'ASK': None,
    # Add more mappings or corrections as necessary
}

# Normalize the base_model column to match keys in custom_mapping (case-insensitive)
df['base_model_normalized'] = df['base_model'].str.lower()

# Function to infer manufacturer from base_model using the custom mapping
def infer_manufacturer_custom(base_model_normalized, custom_mapping4):
    for model, manufacturer in custom_mapping4.items():
        if model.lower() == base_model_normalized:
            return manufacturer
    return None

# Apply the function to fill missing 'manufacturer' values based on the custom mapping
df['manufacturer'] = df.apply(
    lambda row: infer_manufacturer_custom(row['base_model_normalized'], custom_mapping4) if pd.isnull(row['manufacturer']) else row['manufacturer'], 
    axis=1
)

# Drop the temporary normalized base_model column if no longer needed
df.drop('base_model_normalized', axis=1, inplace=True)

In [47]:
# Extend custom_mapping2 with more specific mappings and corrections
custom_mapping5 = {
    # Additions based on identifiable models or brands
    'Blue': 'Blue Bird Corporation',  # Assuming association with Blue Bird buses
    'Thomas': 'Thomas Built Buses',
    'Keystone': 'Keystone RV',
    'Camry': 'Toyota',
    'crysler': 'Chrysler',
    'Chrsler': 'Chrysler',
    'doge': 'Dodge',
    'Infinti': 'Infiniti',
    # Other entries are too generic or need clarification; handle as appropriate
}


# Normalize the base_model column to match keys in custom_mapping (case-insensitive)
df['base_model_normalized'] = df['base_model'].str.lower()

# Function to infer manufacturer from base_model using the custom mapping
def infer_manufacturer_custom(base_model_normalized, custom_mapping5):
    for model, manufacturer in custom_mapping5.items():
        if model.lower() == base_model_normalized:
            return manufacturer
    return None

# Apply the function to fill missing 'manufacturer' values based on the custom mapping
df['manufacturer'] = df.apply(
    lambda row: infer_manufacturer_custom(row['base_model_normalized'], custom_mapping5) if pd.isnull(row['manufacturer']) else row['manufacturer'], 
    axis=1
)

# Drop the temporary normalized base_model column if no longer needed
df.drop('base_model_normalized', axis=1, inplace=True)

## Descriptive Statistics

In [48]:
# Filter the dataframe for rows where 'manufacturer' is missing
missing_manufacturer_df = df[df['manufacturer'].isnull()]

# Count occurrences of each 'base_model' and get the top 10
top_ten_base_models = missing_manufacturer_df['base_model'].value_counts().head(50)

top_ten_base_models

base_model
ALL                  52
All                  46
BUY                  40
Janesville           31
Flexible             30
to                   24
SPECIAL              19
cars                 18
Other                17
Mobility             16
Emergency            15
any                  14
Any                  13
NEW                  13
WOLF                 12
THE                  10
WATER                 9
CHEC                  7
Club                  7
Town                  7
Cars                  7
todos                 7
CLICK                 7
wheelchair            7
Western               6
FABRIQUE              6
ASK                   6
am                    6
2D8HN54159R611084     6
Eagle                 6
F8OO                  5
John                  5
FRHT                  5
SUPER                 5
BMX                   5
Alpha                 5
E350                  4
2010-2017             4
Jayco                 4
AM                    4
Van                   4
hyand

In [49]:
df.shape

(343615, 17)

In [50]:
# Count the number of unique values in each column of the DataFrame
unique_values_count = df.nunique()

# Print the count of unique values for each column
print(unique_values_count)

id                 343615
region                404
price               14630
year                   28
manufacturer           93
fuel                    5
odometer            91379
title_status            6
transmission            3
type                   13
state                  51
posting_year            1
posting_month           2
posting_day            30
posting_weekday         7
posting_hour           24
base_model           3453
dtype: int64


In [51]:
# Calculate the sum of null values in each column
null_values_sum = df.isnull().sum()

# Display the sum of null values for each column
print(null_values_sum)

id                     0
region                 0
price                  0
year                   0
manufacturer        1828
fuel                1887
odometer               0
title_status        6513
transmission        1385
type               70898
state                  0
posting_year           0
posting_month          0
posting_day            0
posting_weekday        0
posting_hour           0
base_model             0
dtype: int64


In [52]:
df.dtypes

id                   int64
region              object
price                int64
year               float64
manufacturer        object
fuel                object
odometer           float64
title_status        object
transmission        object
type                object
state               object
posting_year         int32
posting_month        int32
posting_day          int32
posting_weekday      int32
posting_hour         int32
base_model          object
dtype: object

## 'Type' Imputation

In [53]:
# Assuming you're starting fresh with df2 being a copy of df
df2 = df.copy()

In [54]:
from sklearn.preprocessing import LabelEncoder

encoders = {}  # To store encoders for inverse transformation
categorical_cols = ['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'state', 'base_model']

for col in categorical_cols:
    if df2[col].dtype == 'object':  # Check if column is categorical
        le = LabelEncoder()
        df2[col] = df2[col].fillna('missing')  # Temporarily fill NaNs for encoding
        df2[col] = le.fit_transform(df2[col])
        encoders[col] = le  # Store encoder

In [55]:
# Assuming NaN represents missing values in 'type'
missing_type_indices = df2[df2['type'].isna()].index  # Identify missing values

In [56]:
# Temporarily replace NaN with a placeholder, encode, then map back
df2['type'] = df2['type'].fillna('missing')
type_le = LabelEncoder()
df2['type_encoded'] = type_le.fit_transform(df2['type'])  # Encode 'type'

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Exclude 'missing' type rows for training
training_data = df2[df2['type'] != 'missing']

X = training_data[categorical_cols]  # Features
y = training_data['type_encoded']    # Encoded 'type'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [58]:
if len(missing_type_indices) > 0:
    X_missing = df2.loc[missing_type_indices, categorical_cols]
    predicted_types_encoded = clf.predict(X_missing)
    predicted_types = type_le.inverse_transform(predicted_types_encoded)  # Decode predictions
    
    # Update 'type' in df2 with predicted, decoded values
    df2.loc[missing_type_indices, 'type'] = predicted_types

In [59]:
# Reverse encoding for categorical columns
for col, le in encoders.items():
    df2[col] = le.inverse_transform(df2[col])

# Ensure 'type' column is correctly handled
df2.loc[df2['type'] == 'missing', 'type'] = np.nan  # Optional: Set 'missing' back to NaN if preferred

# Clean up temporary encoding if needed
df2.drop('type_encoded', axis=1, inplace=True)

In [60]:
# Update the 'type' column in the original df with the updated values from df2
df['type'] = df2['type']

In [61]:
df.to_csv(r'C:\Users\skyla\Downloads\vehiclescleaned.csv', index=False)

In [62]:
# Calculate the sum of null values in each column
null_values_sum = df.isnull().sum()

# Display the sum of null values for each column
print(null_values_sum)

id                    0
region                0
price                 0
year                  0
manufacturer       1828
fuel               1887
odometer              0
title_status       6513
transmission       1385
type                  0
state                 0
posting_year          0
posting_month         0
posting_day           0
posting_weekday       0
posting_hour          0
base_model            0
dtype: int64


## PySpark Setup

In [63]:
import os
os.environ["JAVA_HOME"] = "C:/javajdk"  # Use forward slashes or double backslashes
os.environ["SPARK_HOME"] = "C:/spark"   # Use forward slashes or double backslashes

import findspark
findspark.init()

from pyspark.sql import SparkSession

In [64]:
import pyspark
from pyspark.sql import SparkSession
print(pyspark.__version__)

3.1.2


In [65]:
!java -version

java version "1.8.0_401"
Java(TM) SE Runtime Environment (build 1.8.0_401-b10)
Java HotSpot(TM) 64-Bit Server VM (build 25.401-b10, mixed mode)


In [66]:
import os

print("SPARK_HOME:", os.environ.get("SPARK_HOME"))
print("JAVA_HOME:", os.environ.get("JAVA_HOME"))

SPARK_HOME: C:/spark
JAVA_HOME: C:/javajdk


In [67]:
import pkg_resources

# Attempt to get the distribution of py4j
py4j_dist = pkg_resources.get_distribution("py4j")

if py4j_dist:
    print("Installed py4j version:", py4j_dist.version)
else:
    print("py4j is not installed.")

Installed py4j version: 0.10.9


In [68]:
spark = SparkSession.builder.appName("Vehicle Price Prediction").getOrCreate()

In [69]:
# Load the dataset
df = spark.read.option("inferSchema", "true").option("header", "true").csv("file:///C:/Users/skyla/Downloads/vehiclescleaned.csv")
df.show(5)
df.printSchema()

+----------+------+-----+------+------------+----+--------+------------+------------+------+-----+------------+-------------+-----------+---------------+------------+----------+
|        id|region|price|  year|manufacturer|fuel|odometer|title_status|transmission|  type|state|posting_year|posting_month|posting_day|posting_weekday|posting_hour|base_model|
+----------+------+-----+------+------------+----+--------+------------+------------+------+-----+------------+-------------+-----------+---------------+------------+----------+
|7316814884|auburn|33590|2014.0|         gmc| gas| 57923.0|       clean|       other|pickup|   al|        2021|            5|          4|              1|          17|    sierra|
|7316814758|auburn|22590|2010.0|   chevrolet| gas| 71229.0|       clean|       other|pickup|   al|        2021|            5|          4|              1|          17| silverado|
|7316814989|auburn|39590|2020.0|   chevrolet| gas| 19160.0|       clean|       other|pickup|   al|        2021

In [70]:
df = df.dropna()

In [71]:
# Number of rows
num_rows = df.count()
print("Number of rows:", num_rows)

# Column names (equivalent to shape[1] in Pandas)
num_columns = len(df.columns)
print("Number of columns:", num_columns)

Number of rows: 332428
Number of columns: 17


## PySpark Linear Regression

In [72]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

# Import LinearRegression

# Create a LinearRegression estimator
lr = LinearRegression(featuresCol="features", labelCol="price")

# Split the data into training and testing sets
train_data, test_data = df.randomSplit([0.7, 0.3], seed=42)

# Define a list of categorical columns
categorical_columns = ['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'type', 'state', 'base_model']

# Create a StringIndexer for each categorical column
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid="keep") for column in categorical_columns]

# Transform the training data using the indexers
for indexer in indexers:
    train_data = indexer.fit(train_data).transform(train_data)

# Drop the original string columns from the training data
train_data = train_data.drop(*categorical_columns)

# Transform the test data using the indexers
for indexer in indexers:
    test_data = indexer.fit(test_data).transform(test_data)

# Drop the original string columns from the test data
test_data = test_data.drop(*categorical_columns)

# Now, assemble the features into a single vector column
assembler = VectorAssembler(inputCols=train_data.columns[1:], outputCol="features")

# Transform the training data
train_data = assembler.transform(train_data)

# Transform the test data
test_data = assembler.transform(test_data)

# Now, train the linear regression model
model = lr.fit(train_data)

# Once trained, you can use the model to make predictions on the test data
predictions = model.transform(test_data)

# Show some example predictions
predictions.select("prediction", "price", "features").show(5)

+------------------+-----+--------------------+
|        prediction|price|            features|
+------------------+-----+--------------------+
|31589.999956437787|31590|[31590.0,2017.0,2...|
|45590.000006029826|45590|[45590.0,2020.0,3...|
|22989.999897354828|22990|[22990.0,2020.0,3...|
| 25589.99991803942|25590|[25590.0,2018.0,3...|
|20589.999991575594|20590|[20590.0,2013.0,7...|
+------------------+-----+--------------------+
only showing top 5 rows



In [73]:
from pyspark.ml.evaluation import RegressionEvaluator

# Evaluate the model using RMSE
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data =", rmse)

Root Mean Squared Error (RMSE) on test data = 9.068182162941225e-05


## PySpark Decision Tree

In [74]:
from pyspark.ml.regression import DecisionTreeRegressor

# Define the decision tree regressor
dt = DecisionTreeRegressor(featuresCol="features", labelCol="price", maxBins=3000)  # Set maxBins to a larger value

# Train the decision tree model
dt_model = dt.fit(train_data)

# Make predictions on the test data
dt_predictions = dt_model.transform(test_data)

# Evaluate the model using RMSE
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
dt_rmse = evaluator.evaluate(dt_predictions)

print("Decision Tree Root Mean Squared Error (RMSE) on test data =", dt_rmse)

Decision Tree Root Mean Squared Error (RMSE) on test data = 684.843792729947


## PySpark Random Forest

In [75]:
from pyspark.ml.regression import RandomForestRegressor

# Define the Random Forest regressor
rf = RandomForestRegressor(featuresCol="features", labelCol="price", maxBins=3000)

# Train the Random Forest model
rf_model = rf.fit(train_data)

# Make predictions on the test data
rf_predictions = rf_model.transform(test_data)

# Evaluate the model using RMSE
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rf_rmse = evaluator.evaluate(rf_predictions)

print("Random Forest Root Mean Squared Error (RMSE) on test data =", rf_rmse)

Random Forest Root Mean Squared Error (RMSE) on test data = 4003.354263095427
